# TPU Hardware Accelerator

To vastly speed up processing, we can use the TPU available from the Google Colab cloud service.

For a resource on TPUs in Colab, peruse:

https://colab.research.google.com/notebooks/tpu.ipynb

For a simple tutorial, peruse:

https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/classification_iris_data_with_keras.ipynb

For a quick-start guide, peruse:

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/tpu.ipynb

# Import **tensorflow** Library

Import library and alias it:

In [ ]:
import tensorflow as tf

 # Enable TPU Runtime
 
It’s very easy to enable the TPU in a Colab notebook:

1.	click **Runtime** in the top left menu
2.	click **Change runtime** type from the drop-down menu
3.	choose **TPU** from the Hardware accelerator drop-down menu
4.	click **SAVE**

# TPU Detection

Set up TPU resolver and verify that it is running:

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

# Configure TPU

Make devices on the cluster available to use:

In [ ]:
tf.config.experimental_connect_to_cluster(tpu)

Initialize TPU devices:

In [ ]:
tf.tpu.experimental.initialize_tpu_system(tpu)

# Create Strategy

Create a TPU strategy for this notebook:

In [ ]:
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Displayed is a list of available TPUs and other devices.

# Manual Device Placement

After the TPU is initialized, we can use manual device placement to direct computation on a single TPU device:

In [ ]:
a = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
b = [[7.0, 8.0], [9.0, 10.0], [11.0, 12.0]]
with tf.device('/TPU:7'):
  c = tf.matmul(a, b)
I = [[1.0, 0.0], [0.0, 1.0]]
with tf.device('/TPU:6'):
  d = tf.matmul(c, I)   
print('c device:', c.device)
print(c)
print('d device:', d.device)
print(d)

Multiply matrix a by matrix b and place the result in matrix c within the scope of TPU 7. Next, multiply matrix c by the Identity matrix and place the result in matrix d within the scope of TPU 6.

# Run a Computation in all TPU Cores

To replicate a computation so it can run in all TPU cores, pass it to the *strategy.run* API:

In [ ]:
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

z = tpu_strategy.run(matmul_fn, args=(a, b))
print(z)

Create a function that multiplies two matrices. If eager behavior is enabled, make the function a `tf.function` or call `strategy.run` inside a `tf.function`. As of this writing, eager behavior is automatically enabled in Colab!

Invoke the funciton with our `strategy.run` and all the cores obtain the same inputs (a, b) and do the matmul on each core independently. The outputs are the values from all the replicas.

# Digits Data Experiment

The **digits** dataset is embedded in the **sklearn.datasets** package. It consists of 1,797 8x8 grayscale images. Each image is of a hand-written digit from 0-9.

Import libraries:

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Load digits data:

In [ ]:
digits = load_digits()

## Get Keys

Get keys associated with the dataset:

In [ ]:
digits.keys()

## Display an Image

Display the first image from the dataset:

In [ ]:
images = digits.images
image = images[0]
fig = plt.imshow(image, cmap='binary')
fig = plt.axis('off')

## Create a Function to Preprocess Data

The function splits the dataset into train and test sets:

In [ ]:
def load_data(digits, splits, random, scale):
  X = digits.images
  y = digits.target
  x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=splits, random_state=random)
  x_train, x_test = x_train / scale, x_test / scale
  return (x_train, y_train), (x_test, y_test)

## Get Data into Train and Test Sets

Split into 2/3 train and 1/3 test:

In [ ]:
splits, seed, scale = 0.33, 0, 255.0

(x_train, y_train), (x_test, y_test) = load_data(
    digits, splits, seed, scale)

Get target names and number of classes:

In [ ]:
target_names = digits.target_names
num_classes = len(target_names)
num_classes, target_names

## Prepare Data for TensorFlow Consumption

Convert dataset to a TensorFlow consumable form:

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test))

Verify data:

In [ ]:
for img, lbl in train_dataset.take(1):
  print (img.shape, lbl)

## Build the Input Pipeline

Set batch and shuffle size. Build the pipeline:

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_ds = train_dataset\
 .shuffle(SHUFFLE_BUFFER_SIZE)\
 .batch(BATCH_SIZE)
test_ds = test_dataset.batch(BATCH_SIZE)

Get input shape:

In [ ]:
for item in train_ds.take(1):
  s = item[0].shape
in_shape = s[1:]
in_shape

## Build Model

Import libraries:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

Create a function to build the model:

In [ ]:
def get_model():
  return tf.keras.Sequential([
    Flatten(input_shape=in_shape),
    Dense(256, input_shape=in_shape, activation='relu'),
    Dense(num_classes, activation='softmax')])

Create the model and compile within the TPU scope:

In [ ]:
with tpu_strategy.scope():
  model = get_model()
  model.compile(
      optimizer='adam',
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

Creating the model in the TPUStrategy scope means we train the model on the TPU.

Inspect the model:

In [ ]:
model.summary()

## Train Model

Train the model for sixty epochs:

In [ ]:
epochs = 60
history = model.fit(train_ds, epochs=epochs,
                    validation_data=(test_ds))

# MNIST Experiment

Load MNIST train and test sets as TFDS objects:

In [ ]:
import tensorflow_datasets as tfds

train, info = tfds.load(name='mnist', split='train',
                        as_supervised=True, try_gcs=True,
                        with_info=True, shuffle_files=True)
test = tfds.load(name='mnist', split='test',
                        as_supervised=True, try_gcs=True)

## Get Metadata

Display information about the dataset with the **info** object:

In [ ]:
info

## Get Number of Images and Classes

Number of images:

In [ ]:
num_train_img = info.splits['train'].num_examples
num_test_img = info.splits['test'].num_examples
print ('train images:', num_train_img)
print ('test images:', num_test_img)

Number of classes:

In [ ]:
mnist_classes = info.features['label'].num_classes
mnist_classes

## Display Examples

Display some examples with the **show_examples** function:

In [ ]:
fig = tfds.show_examples(train, info)

## Create Scaling Function

Create a function that scales TFDS data:

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32) / 255.0
  return image, label

## Build the Input Pipeline

In [ ]:
BATCH_SIZE = 200
SHUFFLE_SIZE = 10000

train_dataset = train.map(scale)\
  .shuffle(SHUFFLE_SIZE).repeat()\
  .batch(BATCH_SIZE).prefetch(1)
test_dataset = test.map(scale)\
  .batch(BATCH_SIZE).prefetch(1)

Only shuffle and **repeat** the training dataset. The advantage to an infinite dataset for training is to avoid the potential last partial batch in each epoch, so users don't need to think about scaling the gradients based on the actual batch size.

## Create Step Size

When repeating data, include the steps per epoch and validation steps:

In [ ]:
steps_per_epoch = num_train_img // BATCH_SIZE
validation_steps = num_test_img // BATCH_SIZE
steps_per_epoch, validation_steps

## Build Model

Get input shape:

In [ ]:
for item in train_dataset.take(1):
  s = item[0].shape
mnist_shape = s[1:]
mnist_shape

Create function that builds the model:

In [ ]:
def create_model():
  return Sequential([
    Flatten(input_shape=mnist_shape),
    Dense(512, activation='relu'),
    Dense(mnist_classes, activation='softmax')
    ])

Clear previous models and generate seed:

In [ ]:
import numpy as np

tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

## Train within TPU Scope

Import library for the loss function:

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

Within TPU strategy scope, create model and compile:

In [ ]:
with tpu_strategy.scope():
  model = create_model()
  model.compile(
      optimizer='adam',
      steps_per_execution = 50,
      loss=SparseCategoricalCrossentropy(from_logits=True),
      metrics=['sparse_categorical_accuracy'])

Creating the model in the TPUStrategy scope means we train the model on the TPU.
Experiment with `steps_per_execution`. Anything between 2 and `steps_per_epoch` could help here.

Train:

In [ ]:
history = model.fit(
    train_dataset, epochs=5,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps)

# Fashion-MNIST Experiment

**Fashion-MNIST** is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. The dataset is intended to serve as a direct drop-in replacement of the original MNIST dataset for benchmarking machine learning algorithms.

Load Fashion-MNIST as a tf.keras dataset:

In [ ]:
fashion_train, fashion_test = tf.keras.datasets\
  .fashion_mnist.load_data()

Create image and label sets:

In [ ]:
train_img, train_lbl = fashion_train
test_img, test_lbl = fashion_test

Add color (grayscale) dimension:

In [ ]:
fashion_train_img = np.expand_dims(train_img, -1)
fashion_test_img = np.expand_dims(test_img, -1)

Inspect image tensors:

In [ ]:
fashion_train_img.shape, fashion_test_img.shape

Create a function to convert tensors to float:

In [ ]:
def float_it(x):
  return x.astype(np.float32)

In [ ]:
fash_train_img, fash_train_lbl = float_it(
    fashion_train_img), float_it(train_lbl)
fash_test_img, fash_test_lbl = float_it(
    fashion_test_img), float_it(test_lbl)

## Train with TPU Scope

Import libraries:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization,\
  Conv2D, MaxPooling2D, Dropout

Create a function to build the model:

In [ ]:
def create_model():
  return Sequential([
    BatchNormalization(input_shape=(28,28,1)),
    Conv2D(64, (5, 5), padding='same', activation='elu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Dropout(0.25),
    BatchNormalization(),
    Conv2D(128, (5, 5), padding='same', activation='elu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    BatchNormalization(),
    Conv2D(256, (5, 5), padding='same', activation='elu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),    
    Dropout(0.25),
    Flatten(),
    Dense(256, activation='elu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    ])

Clear and seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

Create and compile the model within TPU scope:

In [ ]:
with tpu_strategy.scope():
  model = create_model()
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

## Train the Model

Train the model based on Fashion-MNIST data:

In [ ]:
history = model.fit(fash_train_img, fash_train_lbl,
    epochs=17, steps_per_epoch=60,
    validation_data=(fash_test_img, fash_test_lbl),
    validation_freq=17)

Evaluate:

In [ ]:
loss, acc = model.evaluate(fash_test_img, fash_test_lbl)
print ('loss:', loss)
print ('accuracy:', acc)

## Save the Trained Model

We can preserve the weights from the trained model:

In [ ]:
model.save_weights('./fashion_mnist.h5', overwrite=True)

## Make Inferences

Let’s see how well the model can predict fashion categories!

Get label names:

In [ ]:
class_labels = ['t_shirt', 'trouser', 'pullover', 'dress',
                'coat', 'sandal', 'shirt', 'sneaker',
                'bag', 'ankle_boots']

Create a new model from the saved weights of the trained model:

In [ ]:
new_model = create_model()
new_model.load_weights('./fashion_mnist.h5')

Get 40 prediction arrays from the test set:

In [ ]:
preds = new_model.predict(fash_test_img)[:40]

Transform prediction arrays to predictions:

In [ ]:
pred_40 = [tf.argmax(i).numpy() for i in preds]

Get images and labels for display:

In [ ]:
images, labels = [], []
for i in range(40):
  img = tf.squeeze(fash_test_img[i])
  images.append(img)
  labels.append(int(fash_test_lbl[i]))

Create a function to display predictions:

In [ ]:
def display_test(feature, target, num_images,
                 n_rows, n_cols, cl, p):
  for i in range(num_images):
    plt.subplot(n_rows, 2*n_cols, 2*i+1)
    plt.imshow(feature[i], cmap='nipy_spectral')
    pred = cl[p[i]]
    actual = cl[int(target[i])]
    title_obj = plt.title(actual + ' (' +\
                          pred + ') ')
    if pred == actual:
      title_obj
    else:
      plt.getp(title_obj, 'text')
      plt.setp(title_obj, color='r')
    plt.tight_layout()
    plt.axis('off')

Invoke the function:

In [ ]:
num_rows, num_cols = 10, 4
num_images = num_rows*num_cols
plt.figure(figsize=(20, 20))
display_test(images, labels, num_images, num_rows,
             num_cols, class_labels, pred_40)

# Flowers Experiment

TPUs are very fast. So the training data streaming into the model must keep pace with the training speed of the model to fully leverage the power of TPUs. The preferred method for TPU usage is to store data into the protobuf-based TFRecord format.

The flowers dataset is stored on Google Cloud Storage (GCS). To fully apply the parallelism TPUs offer and to avoid bottlenecking on data transfer, we read data as TFRecord files with approximately 230 images per file. We use tf.data.experimental.AUTOTUNE to optimize different parts of input loading.

A nice site on TPUs in Colab:

https://colab.research.google.com/notebooks/tpu.ipynb

## Read Flowers Data as TFRecord Files

Read TFRecord files:

In [ ]:
piece1 = 'gs://flowers-public/'
piece2 = 'tfrecords-jpeg-192x192-2/*.tfrec'
TFR_GCS_PATTERN = piece1 + piece2
tfr_filenames = tf.io.gfile.glob(TFR_GCS_PATTERN)

Get the number of buckets:

In [ ]:
num_images = len(tfr_filenames)
print ('Pattern matches {} image buckets.'.format(num_images))

Display all 16 buckets (TFRecord files):

In [ ]:
filenames_tfrds = tf.data.Dataset.list_files(TFR_GCS_PATTERN)
for filename in filenames_tfrds.take(16):
  print (filename.numpy())

## Set Parameters for Training Data

Set parameters for image resizing, pipelining, and number of epochs:

In [ ]:
IMAGE_SIZE = [192, 192]
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
SHUFFLE_SIZE = 100
EPOCHS = 9

Set parameters for data splits and labels:

In [ ]:
VALIDATION_SPLIT = 0.19
CLASSES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

Create data splits, validation steps, and steps per epoch:

In [ ]:
split = int(len(tfr_filenames) * VALIDATION_SPLIT)
training_filenames = tfr_filenames[split:]
validation_filenames = tfr_filenames[:split]
print ('Splitting dataset into {} training files and {} '\
        'validation files'\
       .format(len(tfr_filenames), len(training_filenames),
               len(validation_filenames)), end = ' ')
print ('with a batch size of {}.'.format(BATCH_SIZE))

validation_steps = int(3670 // len(tfr_filenames) *\
                       len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(tfr_filenames) *\
                      len(training_filenames)) // BATCH_SIZE
print ('There are {} batches per training epoch and {} '\
       'batches per validation run.'\
       .format(BATCH_SIZE, steps_per_epoch, validation_steps))

## Create Functions to Load and Process TFRecord Files

Create a function to parse a TFRecord file:

In [ ]:
def read_tfrecord(example):
  features = {
      'image': tf.io.FixedLenFeature([], tf.string),
      'class': tf.io.FixedLenFeature([], tf.int64)
  }
  example = tf.io.parse_single_example(example, features)
  image = tf.image.decode_jpeg(example['image'], channels=3)
  image = tf.cast(image, tf.float32) / 255.0 
  image = tf.reshape(image, [*IMAGE_SIZE, 3])
  class_label = example['class']
  return image, class_label

The function accepts an example from a TFRecord file. A dictionary holds datatypes common to TFRecords. The tf.string parameter converts the image to byte strings (list of bytes). The tf.int64 parameter converts the class label to a 64-bit integer scalar value. The example is parsed into (image, label) tuples. The image element, a JPEG-encoded image, is decoded into a uint8 image tensor. The image tensor is scaled to \[0, 1\] range for faster training. It is then reshaped to a standard size for model consumption. The class label element is cast to a scalar. 

Create a function to load TFRecord files as tf.data.Dataset:

In [ ]:
def load_dataset(filenames):
  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False
  dataset = tf.data.TFRecordDataset(
      filenames, num_parallel_reads=AUTO)
  dataset = dataset.with_options(option_no_order)
  dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
  return dataset

The function accepts TFRecord files. For optimal performance, code is included to read from multiple TFRecord files at once. The options setting allows order-altering optimizations. As such, **n** files are read in parallel and data order is disregarded in favor of reading speed.

Create a function to augment training data:

In [ ]:
def data_augment(image, label):
  modified = tf.image.random_flip_left_right(image)
  modified = tf.image.random_saturation(modified, 0, 2)
  return modified, label

Create a function to build an input pipeline from TFRecord files:

In [ ]:
def get_batched_dataset(filenames, train=False):
  dataset = load_dataset(filenames)
  dataset = dataset.cache()
  if train:
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(SHUFFLE_SIZE)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(AUTO)
  return dataset

The function accepts TFRecord files and calls the `load_dataset` function. The function continues by building an input pipeline by caching, repeating, shuffling, batching and prefetching the dataset. Repeating and shuffling are only mapped to training data. We follow best practices by repeating and shuffling only the training data.

## Create Train and Test Sets

Instantiate the datasets:

In [ ]:
training_dataset = get_batched_dataset(
    training_filenames, train=True)
validation_dataset = get_batched_dataset(
    validation_filenames, train=False)
training_dataset, validation_dataset

Display an image:

In [ ]:
for img, lbl in training_dataset.take(1):
  plt.axis('off')
  plt.title(CLASSES[lbl[0].numpy()])
  fig = plt.imshow(img[0])
  tfr_flower_shape = img.shape[1:]

## Model Data

Clear and seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

Create model function:

In [ ]:
def create_model():
  return Sequential([
    Conv2D(32, (3, 3), activation = 'relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='sigmoid')
])

Create and compile model within TPU scope:

In [ ]:
with tpu_strategy.scope():
  flower_model = create_model()
  flower_model.compile(
      optimizer='adam',
      loss=tf.losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])

Train model:

In [ ]:
history = flower_model.fit(training_dataset, epochs=EPOCHS,
                    verbose=1, steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps, 
                    validation_data=validation_dataset)

## Make Inferences

Let’s see how well the model can predict fashion categories!

Grab 40 predictions from the validation (test) set:

In [ ]:
preds = flower_model.predict(validation_dataset)[:40]

Convert prediction arrays to scalar prediction values:

In [ ]:
pred_40 = [tf.argmax(i).numpy() for i in preds]

Create images and labels for display:

In [ ]:
images, labels = [], []
for img, lbl in validation_dataset.take(1):
  for i in range(40):
    actual_img = tf.squeeze(img[i])
    images.append(actual_img)
    labels.append(lbl[i].numpy())

Create a function to display predictions:

In [ ]:
def display_test(feature, target, num_images,
                 n_rows, n_cols, cl, p):
  for i in range(num_images):
    plt.subplot(n_rows, 2*n_cols, 2*i+1)
    plt.imshow(feature[i])
    pred = cl[p[i]]
    actual = cl[int(target[i])]
    title_obj = plt.title(actual + ' (' +\
                          pred + ') ')
    if pred == actual:
      title_obj
    else:
      plt.getp(title_obj, 'text')
      plt.setp(title_obj, color='r')
    plt.tight_layout()
    plt.axis('off')

Invoke function:

In [ ]:
num_rows, num_cols = 10, 4
num_images = num_rows*num_cols
plt.figure(figsize=(20, 20))
display_test(images, labels, num_images, num_rows,
             num_cols, CLASSES, pred_40)